In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(rc={"figure.figsize": (12, 8)})

In [2]:
CSV_PATH = "../../data/csv/"
DF_NAME = "ksou_50095x19.csv"
df = pd.read_csv(CSV_PATH + DF_NAME)

In [3]:
print(df.columns)
df.soldPrice = df.soldPrice.astype(int)

Index(['id', 'address', 'nearbySchoolNum', 'houseType', 'soldPrice', 'bedroom',
       'bathroom', 'carspace', 'soldTimeYear', 'soldTimeMonth', 'landSize',
       'buildingSize', 'buildYear', 'distanceToCBD', 'distanceToStation',
       'pool', 'furnished', 'yard', 'ac'],
      dtype='object')


In [4]:
print(df.soldPrice.describe())
dim = df.shape

count    5.009500e+04
mean     5.362530e+05
std      3.721165e+05
min      5.100000e+04
25%      3.550000e+05
50%      4.590000e+05
75%      6.150000e+05
max      2.000000e+07
Name: soldPrice, dtype: float64


It seems like that the low range is normal and no need to remove outliars from the low range.
But the high range outliars are pretty far away from the value and need to be removed.

The distribution features are:
* Deviate from Normal Distribution
* the positive skewness 
* high peakedness

#### Missing data

In [5]:
def print_missing(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count() * 100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    display(missing_data[missing_data['Percent']>0])
print_missing(df)

,Total,Percent
buildYear,26257,52.414413
buildingSize,16674,33.284759
carspace,4261,8.505839
houseType,3273,6.533586
bathroom,2619,5.228067
bedroom,2557,5.104302
landSize,1120,2.235752
soldTimeMonth,807,1.610939
soldTimeYear,807,1.610939
distanceToStation,203,0.405230


## Relationship between sale price and numerical features

In [6]:
def draw_numerical_scatter(colname, xlim=None):
    df_filtered = df[df[colname]>0]
    data = pd.concat([df_filtered.soldPrice, df_filtered[colname]], axis = 1)
    data.plot.scatter(x = colname, y = 'soldPrice',xlim = xlim)

Firstly, several key missing values need to be addressed. "houseType", "bathroom" and "bedroom" are key features to the house price. Because we have large amount of data, removing these missing entries is fine.

In [7]:
df_remove_missing = df[~pd.isnull(df.houseType)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.bathroom)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.bedroom)]

display(df_remove_missing.shape)
print_missing(df_remove_missing)

(46747, 19)

,Total,Percent
buildYear,23588,50.458853
buildingSize,14498,31.013755
carspace,1523,3.257963
landSize,809,1.730592
soldTimeMonth,761,1.627912
soldTimeYear,761,1.627912
distanceToStation,152,0.325155
distanceToCBD,152,0.325155


It since there is only about very small fraction of data missing for landSize, soldTimeMonth, soldTimeYear, distanceToStation, distanceToCBD, we will move the missing entries for them.

In [8]:
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.distanceToCBD)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.distanceToStation)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.landSize)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.soldTimeMonth)]
df_remove_missing = df_remove_missing[~pd.isnull(df_remove_missing.soldTimeYear)]
display(df_remove_missing.shape)
print_missing(df_remove_missing)

(45074, 19)

,Total,Percent
buildYear,22285,49.440919
buildingSize,13465,29.873098
carspace,1378,3.057195


For entries missing carspace feature, we fill it with 0 according to the meaning of the original webpage. In the webpage, if carspace column is empty, it is very likely that there is 0 carspace provided. 

In [9]:
df_remove_missing.set_value(index = pd.isnull(df_remove_missing.carspace), col = 'carspace', value = 0)
print_missing(df_remove_missing)

,Total,Percent
buildYear,22285,49.440919
buildingSize,13465,29.873098


In [10]:
df = df_remove_missing
df.shape

(45074, 19)

In [11]:
df.to_csv(CSV_PATH + 'ksou_45074x19.csv', index=False)